<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#List-available-GPUs" data-toc-modified-id="List-available-GPUs-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>List available GPUs</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Functions</a></span><ul class="toc-item"><li><span><a href="#Data-Transformations" data-toc-modified-id="Data-Transformations-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Data Transformations</a></span></li><li><span><a href="#Test/Train-split-Techniques" data-toc-modified-id="Test/Train-split-Techniques-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Test/Train split Techniques</a></span></li></ul></li><li><span><a href="#Additional-Pre-Processing" data-toc-modified-id="Additional-Pre-Processing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Additional Pre-Processing</a></span><ul class="toc-item"><li><span><a href="#Deploy-One-Hot-Encoding" data-toc-modified-id="Deploy-One-Hot-Encoding-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Deploy One-Hot-Encoding</a></span></li><li><span><a href="#Create-Feature_Sets" data-toc-modified-id="Create-Feature_Sets-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Create Feature_Sets</a></span></li></ul></li><li><span><a href="#Support-Vector-Machines" data-toc-modified-id="Support-Vector-Machines-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Support Vector Machines</a></span><ul class="toc-item"><li><span><a href="#Perform-Grid-search" data-toc-modified-id="Perform-Grid-search-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Perform Grid search</a></span></li><li><span><a href="#Train-the-models-with-different-time/spatial" data-toc-modified-id="Train-the-models-with-different-time/spatial-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Train the models with different time/spatial</a></span><ul class="toc-item"><li><span><a href="#Batch-Split-k-fold-Cross-Validation" data-toc-modified-id="Batch-Split-k-fold-Cross-Validation-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Batch Split k-fold Cross Validation</a></span></li><li><span><a href="#Sliding-Window-cross-Validation" data-toc-modified-id="Sliding-Window-cross-Validation-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Sliding Window cross Validation</a></span></li><li><span><a href="#Expanding-Window-Split" data-toc-modified-id="Expanding-Window-Split-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Expanding Window Split</a></span></li><li><span><a href="#Start-End-Split" data-toc-modified-id="Start-End-Split-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Start End Split</a></span></li></ul></li><li><span><a href="#Compare-the-results-of-the-different-time/spaitla-bin-combinations" data-toc-modified-id="Compare-the-results-of-the-different-time/spaitla-bin-combinations-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Compare the results of the different time/spaitla bin combinations</a></span></li><li><span><a href="#Analyize-the-models-with-XAI-methods" data-toc-modified-id="Analyize-the-models-with-XAI-methods-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Analyize the models with XAI methods</a></span></li></ul></li><li><span><a href="#Neural-Networks" data-toc-modified-id="Neural-Networks-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Neural Networks</a></span><ul class="toc-item"><li><span><a href="#Grid-Search" data-toc-modified-id="Grid-Search-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Grid Search</a></span></li><li><span><a href="#Train-the-models-with-different-time/spatial-bins-and-different-test/train-spilt-techniques" data-toc-modified-id="Train-the-models-with-different-time/spatial-bins-and-different-test/train-spilt-techniques-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>Train the models with different time/spatial bins and different test/train spilt techniques</a></span><ul class="toc-item"><li><span><a href="#Batch-Split-k-fold-Cross-Validation" data-toc-modified-id="Batch-Split-k-fold-Cross-Validation-7.2.1"><span class="toc-item-num">7.2.1&nbsp;&nbsp;</span>Batch Split k-fold Cross Validation</a></span></li><li><span><a href="#Sliding-Window-cross-Validation" data-toc-modified-id="Sliding-Window-cross-Validation-7.2.2"><span class="toc-item-num">7.2.2&nbsp;&nbsp;</span>Sliding Window cross Validation</a></span></li><li><span><a href="#Expanding-Window-Split" data-toc-modified-id="Expanding-Window-Split-7.2.3"><span class="toc-item-num">7.2.3&nbsp;&nbsp;</span>Expanding Window Split</a></span></li><li><span><a href="#Start-End-Split" data-toc-modified-id="Start-End-Split-7.2.4"><span class="toc-item-num">7.2.4&nbsp;&nbsp;</span>Start End Split</a></span></li></ul></li><li><span><a href="#Compare-the-results-of--the-different-time/spaitla-bin-combinations" data-toc-modified-id="Compare-the-results-of--the-different-time/spaitla-bin-combinations-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>Compare the results of  the different time/spaitla bin combinations</a></span></li><li><span><a href="#Compare-the-different-test/train-split-techniques" data-toc-modified-id="Compare-the-different-test/train-split-techniques-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Compare the different test/train split techniques</a></span></li><li><span><a href="#Aanalyize-the-models-with-XAI-methods" data-toc-modified-id="Aanalyize-the-models-with-XAI-methods-7.5"><span class="toc-item-num">7.5&nbsp;&nbsp;</span>Aanalyize the models with XAI methods</a></span></li><li><span><a href="#Apply-different-NN-architectures" data-toc-modified-id="Apply-different-NN-architectures-7.6"><span class="toc-item-num">7.6&nbsp;&nbsp;</span>Apply different NN architectures</a></span><ul class="toc-item"><li><span><a href="#Convolutional-NN-(CNN)" data-toc-modified-id="Convolutional-NN-(CNN)-7.6.1"><span class="toc-item-num">7.6.1&nbsp;&nbsp;</span>Convolutional NN (CNN)</a></span></li><li><span><a href="#Recurrent-NN-(RNN)" data-toc-modified-id="Recurrent-NN-(RNN)-7.6.2"><span class="toc-item-num">7.6.2&nbsp;&nbsp;</span>Recurrent NN (RNN)</a></span></li><li><span><a href="#Long-Short-Term-Memory-Networks-(LSTM)" data-toc-modified-id="Long-Short-Term-Memory-Networks-(LSTM)-7.6.3"><span class="toc-item-num">7.6.3&nbsp;&nbsp;</span>Long Short-Term Memory Networks (LSTM)</a></span></li><li><span><a href="#Kernalized-NN" data-toc-modified-id="Kernalized-NN-7.6.4"><span class="toc-item-num">7.6.4&nbsp;&nbsp;</span>Kernalized NN</a></span></li><li><span><a href="#Comparison-of-the-results" data-toc-modified-id="Comparison-of-the-results-7.6.5"><span class="toc-item-num">7.6.5&nbsp;&nbsp;</span>Comparison of the results</a></span></li></ul></li></ul></li><li><span><a href="#Compare-SVM-and-NN-models-in-terms-of-predictive-performance-and-computation-time" data-toc-modified-id="Compare-SVM-and-NN-models-in-terms-of-predictive-performance-and-computation-time-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Compare SVM and NN models in terms of predictive performance and computation time</a></span></li></ul></div>

# Imports

In [65]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns


from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold, GridSearchCV, TimeSeriesSplit
from sklearn.compose import TransformedTargetRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.pipeline import Pipeline

from dask_ml.model_selection import GridSearchCV as DaskGridSearchCV
import dask
from dask.distributed import Client

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras.utils import plot_model


from datetime import timedelta
import os
import time

import shap
import lime

import joblib

import math

import tensorflow as tf
import numpy as np
#print("Num GPUs Available:", len([x for x in device_lib.list_local_devices() if x.device_type == 'GPU']))
print("TensorFlow version:", tf.__version__)


print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

tf.config.list_physical_devices('GPU')

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("No GPU found.")
else:
    print(f"GPUs found: {len(gpus)}")
    for gpu in gpus:
        print(f"Device: {gpu.name}")
        print(gpu.device_type)
        details = tf.config.experimental.get_memory_info(gpu.name)

# Load Data

In [77]:
time_periods = [1, 2, 6, 24] # time bins we want to predict the demand for
resolution = ['h3_res_4', 'h3_res_6', 'h3_res_8'] # spatial resolution we want to predict the demand for

prediction_data={}
for period in time_periods:
    res_data={}
    for res in resolution:
        res_data[res]=pd.read_csv(f'../data/{period}hours_{res}.csv', 
                                  parse_dates=['trip_start_timestamp'],
                                  #index_col="trip_start_timestamp"
                                 )
    prediction_data[period]=res_data

In [75]:
prediction_data.get(1).get('h3_res_4')


,trip_start_timestamp,h3_res_4,temperature,precipitation,number_of_trips,weekday,month,hour,hour_sin,hour_cos,weekday_sin,weekday_cos,lagged_1h,lagged_1day
0,2018-01-01 00:00:00,8426645ffffffff,-20.555556,0.0,2,1.0,1.0,0.0,0.000000,1.000000,-2.449294e-16,1.0,NaN,NaN
1,2018-01-01 00:00:00,842664dffffffff,-20.555556,0.0,2321,1.0,1.0,0.0,0.000000,1.000000,-2.449294e-16,1.0,NaN,NaN
2,2018-01-01 00:00:00,8427593ffffffff,-20.555556,0.0,35,1.0,1.0,0.0,0.000000,1.000000,-2.449294e-16,1.0,NaN,NaN
3,2018-01-01 01:00:00,8426645ffffffff,-18.333333,0.0,2,1.0,1.0,1.0,0.269797,0.962917,-2.449294e-16,1.0,2.0,NaN
4,2018-01-01 01:00:00,842664dffffffff,-19.374470,0.0,4192,1.0,1.0,1.0,0.269797,0.962917,-2.449294e-16,1.0,2321.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1372,2018-01-20 13:00:00,8426645ffffffff,6.111111,0.0,4,20.0,1.0,13.0,-0.398401,-0.917211,-2.449294e-16,1.0,5.0,2.0
1373,2018-01-20 13:00:00,842664dffffffff,6.674568,0.0,1828,20.0,1.0,13.0,-0.398401,-0.917211,-2.449294e-16,1.0,1647.0,3201.0
1374,2018-01-20 13:00:00,8427593ffffffff,6.591154,0.0,206,20.0,1.0,13.0,-0.398401,-0.917211,-2.449294e-16,1.0,160.0,173.0
1375,2018-01-20 14:00:00,842664dffffffff,7.222222,0.0,192,20.0,1.0,14.0,-0.631088,-0.775711,-2.449294e-16,1.0,1828.0,3020.0


# Functions

## Data Transformations

In [68]:
def normalize_data(X_train, Y_train, X_test, Y_test):
    '''
    Method that prepares the data for training (split the data/)
    param X: feature data to be prepared
    param Y: target data to be prepared
    param train_index: index that defines the split for trainig data
    param val_index: index that defines the split for target data
    returns X_train, Y_train, X_val, Y_val: prepared training & validation data
    '''
    Scaler=StandardScaler()
    
    X_train = Scaler.fit_transform(X_train)
    Y_train = Scaler.fit_transform(Y_train)
    
    X_test = Scaler.fit_transform(X_test)
    Y_test = Scaler.fit_transform(Y_test)
    
    return X_train, Y_train, X_test, Y_test


def train_nn(X_train, Y_train, X_val, Y_val, model, params):
    '''
    This method compiles and trains a neural network with the given data and parameters
    param X_train: Training data-set
    param Y_train: Target variable for training
    param X_val:   Test data-set
    param y_val:   Target variable for validation
    param model:   NN to be trained
    param params:  Parameters to compile and fit the NN
    returns:       Nothing     
    '''
    model.compile(
        optimizer=params.get("optimizer"),
        loss=params.get("loss"),
        metrics=params.get("metrics"),
    )
    model.fit(
        x=X_train,
        y=Y_train,
        batch_size=params.get("batch_size"),
        epochs=params.get("epochs"),
        #verbose=params.get("verbose"),
    )
    Y_pred=model.predict(X_val)
    r2 = r2_score(Y_val, Y_pred)
    MAE = mean_absolute_error(Y_val, Y_pred)
    print(f"R-squared {r2}")
    print(f"Mean Squared Error {MAE}")
    return None


def train_lsvr(X_train, Y_train, X_val, Y_val, model, params):
    '''
    This method compiles and trains a SVM with the given data and parameters
    param X_train: Training data-set
    param Y_train: Target variable for training
    param X_val:   Test data-set
    param y_val:   Target variable for validation
    param model:   SVM to be trained
    param params:  Parameters to train the SVM
    returns:       Nothing    
    '''
    model.set_params(
              epsilon = params.get('regressor__model__epsilon'),
              C = params.get('regressor__model__C'),
              max_iter = 5000
    )
    model.fit(X_train, 
              Y_train.reshape(len(Y_train))
             )
    Y_pred = model.predict(X_val)
    r2 = r2_score(Y_val, Y_pred)
    MAE = mean_absolute_error(Y_val, Y_pred)
    print(f"R-squared {r2}")
    print(f"Mean Squared Error {MAE}")


def create_date_list(delta):
    '''
    This method creates a list of dates (2018-2019) 
    depending on the delta
    params delta: determines break between dates
    returns:      list of dates 
    '''
    # create a startdate
    start = pd.to_datetime("2018-01-01", format="%Y-%m-%d")
    # create the enddate
    #end = pd.to_datetime("2018-12-31", format="%Y-%m-%d")
    end = pd.to_datetime("2018-01-20", format="%Y-%m-%d")
    # create timedelta to increase days
    next_date = timedelta(days=delta)
    list_dates=[]
    
    while start <= end:
        
        # add date to list
        list_dates.append(start)
        # increase date by one day
        start = start + next_date
    list_dates.append(end)
    
    return list_dates


def create_batch_split(X,Y, date_list, arr):
    '''
    '''
    X_train = X.copy()
    Y_train = Y.copy()
    X_test = pd.DataFrame()
    Y_test = pd.DataFrame()
    # Sort the arr to be able to delete entries with the index
    arr = sorted(arr, reverse=True)
    for element in arr:
        # extract batch
        batch_x = X.loc[(X.index < date_list[element]) & (X.index >= date_list[element] - timedelta(days=7))]
        batch_y = Y.loc[(Y.index < date_list[element]) & (Y.index >= date_list[element] - timedelta(days=7))]
        # add to the test sets
        X_test = pd.concat([batch_x, X_test])
        Y_test = pd.concat([batch_y, Y_test])
        # delete from the training data set
        X_train.drop(index=batch_x.index, inplace=True)
        Y_train.drop(index=batch_y.index, inplace=True)
        # Delete the elements from the date_list that have been already used for the test set
        del date_list[element]
    return X_train, Y_train, X_test, Y_test


def get_columns(df, subset):
    features = list(df.columns)
    if subset == "cos":
        features.remove("weekday")
        features.remove("month")
        features.remove("hour")
    return features

def create_nn(input_size, act_function, decrease_func, factor, first_layer_factor):
    '''
    This method returns a neural network architecture based on the given params
    param input_size:          number of input features
    param act_function:        activation function that should be used for the network
    param decrease:            function decrease of neurons from layer to layer 
                               (determines also number of hidden layers)
    param frist_layer_factor:  determines number of neurons in the first layer (multiplied by input_size)
    returns:                   neural network
    '''
    nn = Sequential()
    neurons = int(input_size * first_layer_factor)
    nn.add(Dense(neurons, input_shape=(input_size,), activation=act_function))
    counter = 1
    while counter >0:
        neurons = decrease_func(factor, input_size, counter)
        if neurons <=1:
            break
        else:
            nn.add(Dense(neurons, activation=act_function))
            counter +=1
    nn.add(Dense(1))
    return nn


def linear_dec_func(factor, neurons, counter):
    '''
    function to determin the number of neurons for the next layer (in this case linear)
    param factor:  slope of the linear function
    param neurons: number of neurons of the last layer
    returns:       number of neruons for the next layer
    '''
    return (int((1-factor *counter) * neurons))
    
def exp_dec_func(factor, neurons, counter):
    '''
    function to determin the number of neurons for the next layer (in this case exponential)
    param factor:  determins pace of decrease
    param neurons: number of neurons of the last layer
    returns:       number of neruons for the next layer
    '''
    return int(math.exp(-counter * factor) * neurons)

## Test/Train split Techniques

In [69]:
def simple_k_fold_validation(k, X, Y, model, train_model, params):
    '''
    Method that trains and validate the model using k-fold validation
    However, this method can disrupt the temporal patterns and data leakage occurs 
    due to the lagged time features
    param k:      Number of folds (iterations)
    param x:      Feature data
    param y:      Target data
    param model:  Model to be trained ()
    returns:      Nothing
    '''
    # initialize the folds
    k_fold = KFold(n_splits= k, random_state=47, shuffle=True)
    # iteratre through all folds
    for train_index, val_index in k_fold.split(X,Y):
        # normalize data
        X_train, Y_train, X_val, Y_val = normalize_data(
            X.iloc[train_index],
            Y.iloc[train_index].values.reshape(-1,1), 
            X.iloc[val_index], 
            Y.iloc[val_index].values.reshape(-1,1)
        )
        # train & validate the model
        train_model(X_train=X_train, Y_train= Y_train, X_val=X_val, Y_val=Y_val,  model=model, params=params)

        
def batch_split_cross_validation(k, time_bin, X, Y, model, train_model, params):
    '''
    This method split the entire dataset into batches. Direct Data Leakage is avoided by disrupting
    the chain of the lagged time feature between test and training set. Batches are cut on week and day level 
    depending on the granularity of the time bins.
    param X:              feature data 
    param Y:              target data
    param k_folds:        number of folds for k-fold validation
    param time_bin:       granularity of the time bins
    returns index_dict:   contains indicies to split for cross validation
    '''
    # if 24 split batches by weeks
    if time_bin == 24:
        date_list = create_date_list(7)    
    # else split by days
    else:
        date_list = create_date_list(1)
    # number of batches that are included in the test set
    size_test_split = len(date_list) // k
    for fold in range(0, k):
        print(fold)
        # pick random choices for the test data batches
        if fold==k-1:
            arr = np.random.choice(range(1, len(date_list)), len(date_list)-1, replace=False)
        else: 
            arr = np.random.choice(range(1, len(date_list)), size_test_split, replace=False)   
        # create training and test set with the batches
        X_train, Y_train, X_test, Y_test = create_batch_split(X,Y, date_list, arr)
        # normalize the data
        X_train, Y_train, X_test, Y_test = normalize_data(
            X_train,
            Y_train,
            X_test,
            Y_test
        )
        # train & validate the model
        train_model(X_train=X_train, Y_train= Y_train, X_val=X_test, Y_val=Y_test,  model=model, params=params)


def sliding_window_cross_validation(X,Y,n_windows, model, train_model, params):
    '''
    This method uses sliding window technique to the split the data. The user specifies the size of the window and
    the model is trained sequentially with each window until the last window. Future data is used to the evaluate the
    model. This way ensures no data leakage into the test set
    '''
    return None


def sorted_train_test_split(X, Y, test_size, model, train_model, params):
    '''
    '''
    # sort the entries in ascending order
    X.sort_index(inplace=True)
    Y.sort_index(inplace=True)
    # get split index
    test_index = int(len(X)*(1-test_size))-1
    # normalize data
    X_train, Y_train, X_val, Y_val = normalize_data(
        X.iloc[:test_index],
        Y.iloc[:test_index].values.reshape(-1,1), 
        X.iloc[test_index:], 
        Y.iloc[test_index:].values.reshape(-1,1)
    )
    # train & validate the model
    train_model(X_train=X_train, Y_train= Y_train, X_val=X_val, Y_val=Y_val,  model=model, params=params)
    
    return None

# Additional Pre-Processing

null_count_column1 = df['lagged_1h'].isnull().sum()
print(f"Null values in 'Column1': {null_count_column1}")
null_count_column1 = df['lagged_1day'].isnull().sum()
print(f"Null values in 'Column1': {null_count_column1}")

## Deploy One-Hot-Encoding

In [78]:
encoder = OneHotEncoder(sparse=False)
# encode the hexagons in dummy variables

for period in time_periods:
    for res in resolution:
        df = prediction_data.get(period).get(res)

        df[f'lagged_1day'].fillna(df['number_of_trips'].mean(), inplace= True)
        if period!=24:
            df[f'lagged_{period}h'].fillna(df['number_of_trips'].mean(), inplace= True)
        
        encoded_data = encoder.fit_transform(df[[res]])
        encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([res]))

        #df.reset_index(drop=False, inplace=True)

        df = pd.concat([df, encoded_df], axis=1).drop(res, axis=1)
        df.set_index("trip_start_timestamp", inplace=True);
        
        prediction_data[period][res] = df

1
h3_res_4 1
Index(['trip_start_timestamp', 'h3_res_4', 'temperature', 'precipitation',
       'number_of_trips', 'weekday', 'month', 'hour', 'hour_sin', 'hour_cos',
       'weekday_sin', 'weekday_cos', 'lagged_1h', 'lagged_1day'],
      dtype='object')
1
h3_res_6 1
Index(['trip_start_timestamp', 'h3_res_6', 'temperature', 'precipitation',
       'number_of_trips', 'weekday', 'month', 'hour', 'hour_sin', 'hour_cos',
       'weekday_sin', 'weekday_cos', 'lagged_1h', 'lagged_1day'],
      dtype='object')
1
h3_res_8 1
Index(['trip_start_timestamp', 'h3_res_8', 'temperature', 'precipitation',
       'number_of_trips', 'weekday', 'month', 'hour', 'hour_sin', 'hour_cos',
       'weekday_sin', 'weekday_cos', 'lagged_1h', 'lagged_1day'],
      dtype='object')
2
h3_res_4 2
Index(['trip_start_timestamp', 'h3_res_4', 'temperature', 'precipitation',
       'number_of_trips', 'weekday', 'month', 'hour', 'hour_sin', 'hour_cos',
       'weekday_sin', 'weekday_cos', 'lagged_2h', 'lagged_1day'],
      

## Create Feature_Sets

# Support Vector Machines

In [35]:
svr_linear = LinearSVR()
svr_kernelized = SVR()

In [36]:
tss = TimeSeriesSplit()
scaler = StandardScaler()

pipeline_lsvr = Pipeline([
    ('scaler', scaler),
    ('model', svr_linear)
])
ttr_lsvr = TransformedTargetRegressor(
    regressor = pipeline_lsvr,
    transformer = scaler
)
pipline_ksvr = Pipeline([
    ('scaler', scaler),
    ('model', svr_kernelized)
])
ttr_ksvr = TransformedTargetRegressor(
    regressor = pipline_ksvr,
    transformer = scaler
)

## Perform Grid search

In [80]:
param_grid_linear = {
    'regressor__model__C': [0.1, 1, 10, 100, 150],
    'regressor__model__epsilon': [0.01, 0.1, 0.5, 1],
}

param_grid_k = {
    'regressor__model__C': [0.1, 1, 10, 100],
    'regressor__model__epsilon': [0.01, 0.1, 0.5, 1],
    #'regressor__model__kernel': ['rbf','sigmoid', 'poly'],
    'regressor__model__kernel': ['rbf', 'sigmoid'],
    'regressor__model__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1],
    'regressor__model__coef0': [0, 0.1, 0.5, 1],  # Relevant for 'poly' and 'sigmoid' kernels
    #'regressor__model__degree': [2, 3, 4]  # Only relevant for 'poly' kernel

}

In [81]:
gs_linear_svr = DaskGridSearchCV(
    estimator=ttr_lsvr, 
    param_grid=param_grid_linear, 
    cv=5, 
    scoring='r2',
    n_jobs=-1,
)
gs_ksvr = DaskGridSearchCV(
    estimator=ttr_ksvr, 
    param_grid=param_grid_k, 
    cv=5, 
    scoring='r2',
    n_jobs=3,
)

In [ ]:
        start_time = time.time() # time computation time for the gridsearch
        warnings.filterwarnings('ignore', message='Liblinear failed to converge, increase the number of iterations.')
        gs_linear_svr.fit(df[get_columns(df,'cos')], df['number_of_trips'])
        end_time = time.time()
        
        print(f"GridSearch took: {(end_time-start_time)/60} minuts")

In [21]:
try:
    with Client() as client:
        
        num_cores = client.ncores()
        print(f"Number of cores: {num_cores}")
        print(f"Dashboard link: {client.dashboard_link}")

        start_time = time.time() # time computation time for the gridsearch
        warnings.filterwarnings('ignore', message='Liblinear failed to converge, increase the number of iterations.')
        gs_linear_svr.fit(df[get_columns(df,'cos')], df['number_of_trips'])
        end_time = time.time()

except Exception as e:
    print(f"Exception {e} occured, swtich to running only one job")
    gs_linear_svr.n_jobs=1
    
    start_time = time.time() # time computation time for the gridsearch
    warnings.filterwarnings('ignore', message='Liblinear failed to converge, increase the number of iterations.')
    gs_linear_svr.fit(df[get_columns(df,'cos')], df['number_of_trips'])
    end_time = time.time()

print(f"GridSearch took: {(end_time-start_time)/60} minuts")

Number of cores: {'tcp://127.0.0.1:58056': 2, 'tcp://127.0.0.1:58057': 2, 'tcp://127.0.0.1:58058': 2, 'tcp://127.0.0.1:58065': 2}
Dashboard link: http://127.0.0.1:8787/status
GridSearch took: 0.707597815990448 minuts


In [ ]:
try:
    with Client() as client:
        
        num_cores = client.ncores()
        print(f"Number of cores: {num_cores}")
        print(f"Dashboard link: {client.dashboard_link}")

        start_time = time.time() # time computation time for the gridsearch
        gs_ksvr.fit(df[get_columns(df,'cos')], df['number_of_trips'])
        end_time = time.time()
    
except Exception as e:
    print(f"Exception {e} occured, swtich to running only one job")
    gs_ksvr.n_jobs=1
    
    start_time = time.time() # time computation time for the gridsearch
    gs_ksvr.fit(df[get_columns(df,'cos')], df['number_of_trips'])
    end_time = time.time()

print(f"GridSearch took: {(end_time-start_time)/60} minuts")

Task was destroyed but it is pending!
task: <Task pending coro=<ServerSession.with_document_locked() running at C:\Users\lukas\miniconda3\envs\tf\lib\site-packages\bokeh\server\session.py:89> wait_for=<Future finished result=<tornado.lock...0026A6E27E6C8>> cb=[multi_future.<locals>.callback() at C:\Users\lukas\miniconda3\envs\tf\lib\site-packages\tornado\gen.py:520]>


Number of cores: {'tcp://127.0.0.1:58132': 2, 'tcp://127.0.0.1:58133': 2, 'tcp://127.0.0.1:58134': 2, 'tcp://127.0.0.1:58139': 2}
Dashboard link: http://127.0.0.1:8787/status


In [ ]:
print("SVR with linear kernel")
print(f"best params: {gs_linear_svr.best_params_}")
print(f"best score:  {gs_linear_svr.best_score_}")

print("\nKernelized SVR")
print(f"best params: {gs_ksvr.best_params_}")
print(f"best score:  {gs_ksvr.best_score_}")

In [ ]:
joblib.dump(gs_ksvr.best_estimator_, 'Models/gs_ksvr_model.pkl')
joblib.dump(gs_ksvr.best_estimator_, 'Models/gs_ksvr_params.pkl')

joblib.dump(gs_ksvr.best_estimator_, 'Models/gs_linear_svr_model.pkl')
joblib.dump(gs_ksvr.best_estimator_, 'Models/gs_linear_svr_params.pkl')

## Train the models with different time/spatial

In [ ]:
#Load the best models and params from the gridsearch

# load the best models
joblib.load('Models/gs_linear_svr_model.pkl')
joblib.load('Models/gs_ksvr_model.pkl')

# load the best params
joblib.load('Models/gs_ksvr_params.pkl')
joblib.load('Models/gs_linear_svr_params.pkl')

### Batch Split k-fold Cross Validation

### Sliding Window cross Validation

### Expanding Window Split

### Start End Split

##  Compare the results of the different time/spaitla bin combinations

## Analyize the models with XAI methods

# Neural Networks

In [30]:
nn_model = Sequential()
nn_model.add(Dense(12, input_shape=(13,), activation='relu'))
nn_model.add(Dense(8, activation='relu'))
nn_model.add(Dense(4, activation='relu'))
nn_model.add(Dense(1))
nn_model.save_weights('initial_weights')

In [27]:
params={
    "optimizer":"RMSPROP",
    "loss":"mse",
    "loss_weights":None,
    "metrics":['mae'],
    "weighted_metrics":None,
    "run_eagerly":False,
    #"steps_per_execution":1,
    "jit_compile":"auto",
    "auto_scale_loss":True,
    "x":None,
    "y":None,
    "batch_size":50,
    "epochs":200,
    "verbose":1,
    "callbacks":None,
    "validation_split":0.0,
    "validation_data":None,
    "shuffle":True,
    "class_weight":None,
    "sample_weight":None,
    "initial_epoch":0,
    "steps_per_epoch":None,
    "validation_steps":None,
    #"validation_batch_size":None,
    "validation_freq":1
}

## Grid Search

## Train the models with different time/spatial bins and different test/train spilt techniques

### Batch Split k-fold Cross Validation

In [ ]:
nn_model.load_weights('initial_weights')
batch_split_cross_validation(3, 24, df[features], df[['number_of_trips']], nn_model, train_nn, params)

### Sliding Window cross Validation

### Expanding Window Split

### Start End Split

In [79]:
for period in time_periods:
    for res in resolution:
        df = prediction_data.get(period).get(res)
        columns = get_columns(df, "cos")
        nn = create_nn(len(columns), 'relu', exp_dec_func, 0.8, 1)
        sorted_train_test_split(df[columns], df[['number_of_trips']], 0.1, nn, train_nn, params=params)

h3_res_4 1
temperature                 0
precipitation               0
number_of_trips             0
hour_sin                    0
hour_cos                    0
weekday_sin                 0
weekday_cos                 0
lagged_1h                   0
lagged_1day                 0
h3_res_4_8426641ffffffff    0
h3_res_4_8426645ffffffff    0
h3_res_4_842664dffffffff    0
h3_res_4_8427593ffffffff    0
dtype: int64
Epoch 1/200
1238/1238 [==============================] - 0s 160us/sample - loss: 1.0285 - mean_absolute_error: 0.7334
Epoch 2/200
1238/1238 [==============================] - 0s 29us/sample - loss: 0.9335 - mean_absolute_error: 0.6914
Epoch 3/200
1238/1238 [==============================] - 0s 35us/sample - loss: 0.8796 - mean_absolute_error: 0.6424
Epoch 4/200
1238/1238 [==============================] - 0s 29us/sample - loss: 0.8288 - mean_absolute_error: 0.5901
Epoch 5/200
1238/1238 [==============================] - 0s 32us/sample - loss: 0.7823 - mean_absolute_error: 0.5378


Epoch 67/200
1238/1238 [==============================] - 0s 44us/sample - loss: 0.1291 - mean_absolute_error: 0.1251
Epoch 68/200
1238/1238 [==============================] - 0s 43us/sample - loss: 0.1245 - mean_absolute_error: 0.1217
Epoch 69/200
1238/1238 [==============================] - 0s 31us/sample - loss: 0.1198 - mean_absolute_error: 0.1191
Epoch 70/200
1238/1238 [==============================] - 0s 33us/sample - loss: 0.1150 - mean_absolute_error: 0.1156
Epoch 71/200
1238/1238 [==============================] - 0s 31us/sample - loss: 0.1104 - mean_absolute_error: 0.1133
Epoch 72/200
1238/1238 [==============================] - 0s 35us/sample - loss: 0.1062 - mean_absolute_error: 0.1108
Epoch 73/200
1238/1238 [==============================] - 0s 32us/sample - loss: 0.1020 - mean_absolute_error: 0.1076
Epoch 74/200
1238/1238 [==============================] - 0s 29us/sample - loss: 0.0979 - mean_absolute_error: 0.1050
Epoch 75/200
1238/1238 [==============================] 

Epoch 1/200
7256/7256 [==============================] - 0s 44us/sample - loss: 1.0172 - mean_absolute_error: 0.4059
Epoch 2/200
7256/7256 [==============================] - 0s 21us/sample - loss: 0.9988 - mean_absolute_error: 0.3898
Epoch 3/200
7256/7256 [==============================] - 0s 22us/sample - loss: 0.6352 - mean_absolute_error: 0.3285
Epoch 4/200
7256/7256 [==============================] - 0s 23us/sample - loss: 0.0254 - mean_absolute_error: 0.1281
Epoch 5/200
7256/7256 [==============================] - 0s 22us/sample - loss: 0.0076 - mean_absolute_error: 0.0600
Epoch 6/200
7256/7256 [==============================] - 0s 22us/sample - loss: 0.0057 - mean_absolute_error: 0.0495
Epoch 7/200
7256/7256 [==============================] - 0s 21us/sample - loss: 0.0045 - mean_absolute_error: 0.0421
Epoch 8/200
7256/7256 [==============================] - 0s 23us/sample - loss: 0.0028 - mean_absolute_error: 0.0289
Epoch 9/200
7256/7256 [==============================] - 0s 24us

Epoch 136/200
7256/7256 [==============================] - 0s 27us/sample - loss: 3.0746e-04 - mean_absolute_error: 0.0079
Epoch 137/200
7256/7256 [==============================] - 0s 26us/sample - loss: 2.7769e-04 - mean_absolute_error: 0.0075
Epoch 138/200
7256/7256 [==============================] - 0s 26us/sample - loss: 3.0831e-04 - mean_absolute_error: 0.0082
Epoch 139/200
7256/7256 [==============================] - 0s 26us/sample - loss: 3.7781e-04 - mean_absolute_error: 0.0079
Epoch 140/200
7256/7256 [==============================] - 0s 30us/sample - loss: 2.7226e-04 - mean_absolute_error: 0.0076
Epoch 141/200
7256/7256 [==============================] - 0s 33us/sample - loss: 3.1575e-04 - mean_absolute_error: 0.0077
Epoch 142/200
7256/7256 [==============================] - 0s 30us/sample - loss: 2.9964e-04 - mean_absolute_error: 0.0075
Epoch 143/200
7256/7256 [==============================] - 0s 28us/sample - loss: 3.7366e-04 - mean_absolute_error: 0.0078
Epoch 144/200
72

Epoch 1/200
26689/26689 [==============================] - 2s 66us/sample - loss: 1.0001 - mean_absolute_error: 0.4526
Epoch 2/200
26689/26689 [==============================] - 2s 64us/sample - loss: 1.0001 - mean_absolute_error: 0.4470
Epoch 3/200
26689/26689 [==============================] - 2s 59us/sample - loss: 1.0001 - mean_absolute_error: 0.4512
Epoch 4/200
26689/26689 [==============================] - 2s 66us/sample - loss: 1.0000 - mean_absolute_error: 0.4513
Epoch 5/200
26689/26689 [==============================] - 2s 62us/sample - loss: 1.0001 - mean_absolute_error: 0.4460
Epoch 6/200
26689/26689 [==============================] - 2s 70us/sample - loss: 1.0001 - mean_absolute_error: 0.4517
Epoch 7/200
26689/26689 [==============================] - 2s 64us/sample - loss: 1.0001 - mean_absolute_error: 0.4494
Epoch 8/200
26689/26689 [==============================] - 2s 76us/sample - loss: 1.0001 - mean_absolute_error: 0.4519
Epoch 9/200
26689/26689 [=======================

KeyboardInterrupt: 

## Compare the results of  the different time/spaitla bin combinations 

## Compare the different test/train split techniques

## Aanalyize the models with XAI methods

## Apply different NN architectures

### Convolutional NN (CNN)

### Recurrent NN (RNN)

### Long Short-Term Memory Networks (LSTM)

### Kernalized NN

### Comparison of the results

# Compare SVM and NN models in terms of predictive performance and computation time